## Síntese de Grades de Dammann por GS (Y-pol) com FFT no Espaço-k | Synthesis of Dammann Gratings by GS (Y-pol) with FFT in k-space

**PT — Objetivo**  
Este notebook implementa a geração de uma **supercélula de fase** via **Gerchberg–Saxton (GS)** em **espaço-k** para produzir uma grade de difração tipo **Dammann** (uniformidade de intensidades nos picos).  
A supercélula 2D é otimizada por GS para que o **far-field (FFT)** apresente uma **máscara-alvo** (círculo limitado por NA/λ). Em seguida, a supercélula é **repetida (tiling)** para formar a metassuperfície completa, e calculamos **eficiência de difração (DE)**, **uniformidade (RMSE)** e **mapas**.

Fluxo:  
1) Definir parâmetros físicos e de amostragem;  
2) Construir eixos de frequências espaciais (k-space);  
3) Definir **alvo em k-space** (disco limitado);  
4) Rodar **GS** na supercélula (amplitude alvo no far-field, fase livre);  
5) Extrair **mapa de fase** da supercélula e montar **metassuperfície** (tiling);  
6) Calcular **far-field** global, ordens propagantes (p, q), **DE** e **RMSE**;  
7) Plotar e **salvar** figuras/artefatos com nomes rastreáveis.

---

<details>
<summary><b>Show English</b></summary>

**EN — Goal**  
This notebook implements **phase supercell synthesis** via **Gerchberg–Saxton (GS)** in **k-space** to produce **Dammann-like diffraction grids** (uniform spot intensities).  
We optimize a 2D supercell so that its **far-field (FFT)** matches a **target mask** (disc limited by NA/λ). The supercell is then **tiled** to build the full metasurface. We compute **diffraction efficiency (DE)**, **uniformity (RMSE)** and **maps**.

Steps:  
1) Set physical/sampling parameters;  
2) Build spatial-frequency axes (k-space);  
3) Define **k-space target** (limited disk);  
4) Run **GS** on the supercell (target amplitude in far-field, free phase);  
5) Extract **supercell phase** and build the **metasurface** (tiling);  
6) Compute **global far-field**, propagating orders (p, q), **DE** and **RMSE**;  
7) Plot and **save** figures/artifacts with traceable filenames.

</details>



### 1. 🔧 Imports

**PT — O que usamos e por quê**

- **NumPy (`numpy`)**: operações vetoriais/matriciais e álgebra de Fourier (base para FFT).  
- **Matplotlib (`matplotlib.pyplot`)**: geração de figuras e gráficos (mapas de fase, evolução do erro, etc.).  
- **Pandas (`pandas`)**: tabelas e relatórios (ordens de difração e intensidades).  
- **Pathlib (`pathlib.Path`)**: manipulação de caminhos de forma **portável** (Windows/Linux/macOS).  
- **Datetime (`datetime`)**: criação de um **carimbo de tempo UTC** para organizar “rodadas” (runs) de experimento em diretórios separados.

> Usar `Path` + `datetime` garante que os artefatos sejam salvos com nomes previsíveis e em uma hierarquia limpa, sem depender do “current working directory”.

---

<details>
<summary><b>Show English</b></summary>

**EN — What we use and why**

- **NumPy (`numpy`)**: vector/matrix ops and Fourier algebra.  
- **Matplotlib (`matplotlib.pyplot`)**: figures and charts (phase maps, error curves, etc.).  
- **Pandas (`pandas`)**: tables/reports (diffracted orders and intensities).  
- **Pathlib (`pathlib.Path`)**: **portable** path handling across OSes.  
- **Datetime (`datetime`)**: **UTC timestamps** to isolate experimental runs in distinct folders.

> Using `Path` + `datetime` yields predictable filenames and a clean hierarchy, independent of the current working directory.

</details>


In [4]:
# =============================================================================
# PT: Importações principais
# EN: Main imports
# =============================================================================
import numpy as np                        # PT: computação numérica  | EN: numerical computing
import matplotlib.pyplot as plt           # PT: gráficos             | EN: plotting
import pandas as pd                       # PT: tabelas/relatórios   | EN: tabular reports

# PT: Utilitários para caminhos e timestamp (runs reprodutíveis e organizados)
# EN: Utilities for paths and timestamps (reproducible, organized runs)
from pathlib import Path
from datetime import datetime, timezone


### 2. ⚙️ Parâmetros do Experimento e Organização dos Artefatos | Experiment Parameters and Organization of Artifacts

**PT — Parâmetros físicos e de amostragem**

- `P` — passo (pitch) do **pixel** em **metros**;  
- `wavelength` — comprimento de onda **λ** (m);  
- `supercell_pixels` — tamanho (lado) da **supercélula** em **pixels**;  
- `n_supercells` — número de supercélulas **por lado** no mosaico;  
- `iters_gs` — **iterações** do algoritmo **GS**;  
- `random_seed` — semente global do gerador aleatório (reprodutibilidade).

**PT — Organização dos resultados**

- Definimos a **raiz do repositório** automaticamente (`find_repo_root`) para que o notebook funcione
  mesmo se executado a partir de subpastas.  
- Os resultados são salvos em  
  `results/holography/gs_y/<EXPERIMENT>/<RUN_ID>/`,  
  onde `<RUN_ID>` é um carimbo de tempo em UTC (ISO-like).  
- Arquivos recebem **nomes ricos** via `rich_name(…)`, incluindo `λ`, `P`, `supercell_pixels`, `n_supercells`, `iters_gs` e `random_seed` — isso facilita a rastreabilidade entre parâmetros e saídas.

> Dica: padronizar nomes e pastas evita conflitos e torna fácil comparar rodadas.

---

<details>
<summary><b>Show English</b></summary>

**EN — Physical and sampling parameters**

- `P` — pixel **pitch** (m);  
- `wavelength` — optical **λ** (m);  
- `supercell_pixels` — **supercell** side length (px);  
- `n_supercells` — number of supercells **per side** (tiling);  
- `iters_gs` — **GS** iteration count;  
- `random_seed` — global RNG seed (reproducibility).

**EN — Output organization**

- We auto-detect the **repository root** (`find_repo_root`) so the notebook works regardless of the current folder.  
- Results go under  
  `results/holography/gs_y/<EXPERIMENT>/<RUN_ID>/`,  
  where `<RUN_ID>` is a UTC timestamp.  
- We use **rich filenames** (`rich_name(…)`) that encode `λ`, `P`, `supercell_pixels`, `n_supercells`, `iters_gs`, and `random_seed` for easy traceability.

> Tip: consistent names and folders prevent collisions and simplify run-to-run comparisons.

</details>


In [5]:
# =============================================================================
# PT: Parâmetros do experimento
# EN: Experiment parameters
# =============================================================================
P = 520e-9            # EN: pixel pitch [m]           | PT: passo do pixel [m]
wavelength = 1064e-9  # EN: wavelength λ [m]          | PT: comprimento de onda λ [m]
supercell_pixels = 45 # EN: supercell side (px)       | PT: lado da supercélula (px)
n_supercells = 10     # EN: supercells per side        | PT: nº de supercélulas por lado
iters_gs = 400        # EN: GS iterations              | PT: iterações do GS
random_seed = 0       # EN: RNG seed (reproducibility) | PT: semente do gerador (reprodutibilidade)

# =============================================================================
# PT/EN: Funções utilitárias de organização e nomenclatura
# =============================================================================
def find_repo_root(start: Path = Path.cwd()) -> Path:
    """
    EN:
        Walk upwards from `start` until a directory containing `.git` is found,
        then return that directory. If none is found, return `start`.
        This allows notebooks to resolve paths relative to the repository root,
        regardless of the current working directory.

    PT:
        Sobe a partir de `start` até encontrar um diretório contendo `.git` e
        retorna esse diretório. Se não encontrar, retorna `start`.
        Isso permite que os notebooks resolvam caminhos relativos à raiz do
        repositório, independentemente do diretório de execução atual.
    """
    for parent in [start, *start.parents]:
        if (parent / ".git").exists():
            return parent
    return start

# PT/EN: Descoberta da raiz e montagem dos diretórios de saída
REPO_ROOT = find_repo_root()
EXPERIMENT = "demo_dammannY"  # PT: rótulo lógico do estudo | EN: logical study label
OUT_ROOT   = REPO_ROOT / "results" / "holography" / "gs_y"

# PT/EN: RUN_ID único (UTC) para isolar cada rodada e evitar sobrescritas
run_id  = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H-%M-%SZ")
RUN_DIR = OUT_ROOT / EXPERIMENT / run_id
RUN_DIR.mkdir(parents=True, exist_ok=True)

def _nm(x_m: float) -> int:
    """
    EN:
        Convert meters to nanometers (rounded int) for compact filenames.

    PT:
        Converte metros para nanômetros (inteiro arredondado) para nomes
        de arquivo compactos.
    """
    return int(round(float(x_m) * 1e9))

def rich_name(stem: str) -> str:
    """
    EN:
        Build a descriptive filename stem carrying the key physical parameters and
        reproducibility knobs. Example:
        mapas_fase_difracao__Y__λ_1064nm__P_520nm__scpix_45px__nsc_10__iter_400__seed_0

    PT:
        Monta um prefixo de nome descritivo com os principais parâmetros físicos e
        chaves de reprodutibilidade. Exemplo:
        mapas_fase_difracao__Y__λ_1064nm__P_520nm__scpix_45px__nsc_10__iter_400__seed_0
    """
    return (f"{stem}"
            f"__Y"
            f"__λ_{_nm(wavelength)}nm"
            f"__P_{_nm(P)}nm"
            f"__scpix_{supercell_pixels}px"
            f"__nsc_{n_supercells}"
            f"__iter_{iters_gs}"
            f"__seed_{random_seed}")


### 3. Dimensões e Escalas da Supercélula | Dimensions and scales

**PT — Ideia geral**  
Definimos os tamanhos **em pixels** e os passos **físicos** (em metros) que relacionam o domínio discreto da imagem com o espaço físico do holograma:

- `N_super` — lado da **supercélula** (em pixels), igual a `supercell_pixels`;  
- `N_total` — lado da **metassuperfície completa** (em pixels), obtida ao “tijolar” `n_supercells × n_supercells` supercélulas;  
- `dx` — **pitch físico** por pixel (m), igual a `P`;  
- `d` — **período físico** da supercélula (m), dado por `dx × N_super`.

> Esses termos conectam a malha discreta (px) ao mundo físico (m), e serão usados na construção das frequências espaciais e das ordens de difração.

---

<details>
<summary><b>Show English</b></summary>

**EN — Big picture**  
We define the **pixel-level** sizes and the **physical** sampling:

- `N_super` — **supercell** side (px), equals `supercell_pixels`;  
- `N_total` — **full metasurface** side (px), built by tiling `n_supercells × n_supercells` supercells;  
- `dx` — **physical pixel pitch** (m), equals `P`;  
- `d` — **supercell physical period** (m), `dx × N_super`.

> These terms bridge the discrete grid (px) with the physical world (m) and will be used for spatial frequencies and diffraction orders.

</details>


In [6]:
# =============================================================================
# PT: Dimensões e escalas
# EN: Dimensions and scales
# =============================================================================

# PT: Semente global para reprodutibilidade (gera o mesmo campo inicial aleatório)
# EN: Global seed for reproducibility (yields the same initial random field)
np.random.seed(random_seed)

# PT: Tamanhos em pixels
# EN: Sizes in pixels
N_super = supercell_pixels                  # PT: pixels por supercélula | EN: pixels per supercell
N_total = N_super * n_supercells            # PT: lado total da metassuperfície (px) | EN: full metasurface side (px)

# PT: Passos/Períodos físicos (metros)
# EN: Physical sampling and period (meters)
dx = P                                      # PT: pitch físico por pixel | EN: physical pixel pitch
d  = dx * N_super                           # PT: período físico da supercélula | EN: supercell physical period


### 4. Grades de Frequências Espaciais (k-space) | Spatial frequency grids

**PT — Construção do k-space (supercélula)**  
Criamos os eixos de **frequência espacial** com `np.fft.fftfreq` e os **centralizamos** com `fftshift`.  
A seguir, montamos a malha 2D $(K_x, K_y)$ e o **raio** $K_{\text{rad}} = \sqrt{K_x^2 + K_y^2}$, que nos permite definir **regiões-alvo circulares** no domínio de Fourier.

> Em termos físicos, frequências espaciais delimitam os ângulos de difração acessíveis; a centralização (shift) alinha o DC ao centro da matriz.

---

<details>
<summary><b>Show English</b></summary>

**EN — Building the supercell k-space**  
We construct **spatial frequency** axes via `np.fft.fftfreq`, **center** them using `fftshift`, and form the 2D mesh $(K_x, K_y)$.  
The **radius** $K_{\text{rad}} = \sqrt{K_x^2 + K_y^2}$ is convenient to define **circular targets** in Fourier space.

> Physically, spatial frequencies map to accessible diffraction angles; shifting places DC at the matrix center.

</details>


In [7]:
# =============================================================================
# PT: Grades de frequências espaciais (k-space) para a supercélula
# EN: Spatial frequency grids (k-space) for the supercell
# =============================================================================

# PT: Eixos de frequência espacial (ciclos por metro) e centralização (DC no centro)
# EN: Spatial frequency axes (cycles per meter) and centering (DC in the middle)
kx = np.fft.fftfreq(N_super, d=dx)
ky = np.fft.fftfreq(N_super, d=dx)
kx_shift = np.fft.fftshift(kx)
ky_shift = np.fft.fftshift(ky)

# PT: Malha 2D de frequências espaciais e raio em k
# EN: 2D mesh of spatial frequencies and radial coordinate
KX, KY = np.meshgrid(kx_shift, ky_shift)
K_rad = np.sqrt(KX**2 + KY**2)


### 5. Alvo Circular no Espaço-k (cone angular acessível) | Fourier-domain target

**PT — Máscara circular de amplitude**  
Definimos um **alvo** no domínio de Fourier como um **disco uniforme** de raio  
$K_{\text{cut}} = \min\!\big( 1/\lambda,\; 1/(2\,dx) \big)$,  
que representa um **cone angular acessível** (limitado por **λ** e pela amostragem **dx**).

- Pontos com $K_{\text{rad}} \le K_{\text{cut}}$ recebem amplitude **1** (permitidos).  
- Pontos fora do disco recebem **0** (bloqueados).

> Essa máscara implementa de forma simples uma limitação equivalente a **NA/λ** e Nyquist.

---

<details>
<summary><b>Show English</b></summary>

**EN — Circular amplitude target**  
We define a **uniform disk** in Fourier space with radius  
$K_{\text{cut}} = \min\!\big( 1/\lambda,\; 1/(2\,dx) \big)$,  
representing the **accessible angular cone** (limited by **λ** and sampling **dx**).

- Points with $K_{\text{rad}} \le K_{\text{cut}}$ get amplitude **1**;  
- Outside points get **0**.

> This simple mask approximates an **NA/λ**-like cutoff and Nyquist constraints.

</details>


In [8]:
# =============================================================================
# PT: Alvo no espaço-k (disco uniforme)
# EN: Fourier-domain target (uniform disk)
# =============================================================================

# PT/EN: Raio de corte em k: limitado por λ e por Nyquist (1/(2*dx))
target_radius = min(1.0 / wavelength, 1.0 / (2.0 * dx))

# PT/EN: Máscara circular de amplitude (1 dentro do raio, 0 fora)
target_amp = (K_rad <= target_radius).astype(float)


### 6. GS na Supercélula (fase-única) | GS over the supercell (phase-only)

**PT — Fluxo do laço GS (na supercélula)**

1. **FFT** do campo (far-field da supercélula);  
2. Cálculo do **erro RMSE** entre a amplitude corrente e o alvo `target_amp`;  
3. **Imposição de restrição**: substitui amplitude pela do alvo (disco uniforme), **mantém a fase**;  
4. **IFFT** para voltar ao plano da supercélula;  
5. Impõe **amplitude unitária** (fase-only) no plano da supercélula.

> Ao final, a fase da supercélula distribui energia **uniformemente** no disco do far-field.

---

<details>
<summary><b>Show English</b></summary>

**EN — GS loop over the supercell**

1. **FFT** of the field (supercell far-field);  
2. **RMSE** between current amplitude and `target_amp`;  
3. **Constraint**: replace amplitude by the target (uniform disk), **keep phase**;  
4. **IFFT** back to the supercell plane;  
5. Enforce **unit amplitude** (phase-only) at the supercell plane.

> The final supercell phase yields a **uniform** energy distribution within the disk.

</details>


In [9]:
# =============================================================================
# PT: GS na supercélula (fase-única)
# EN: GS over the supercell (phase-only)
# =============================================================================

# PT: Campo inicial aleatório com fase uniforme em [0, 2π)
# EN: Initial random field with uniform phase in [0, 2π)
plane_field = np.exp(1j * 2.0 * np.pi * np.random.rand(N_super, N_super))

# PT/EN: Histórico do erro (RMSE) por iteração
errors = []

for it in range(iters_gs):
    # ---------------------------------------------------------
    # PT: FFT -> far-field da supercélula (centralizado)
    # EN: FFT -> supercell far-field (centered)
    far = np.fft.fft2(plane_field)
    far_shift = np.fft.fftshift(far)

    # ---------------------------------------------------------
    # PT: Erro RMSE entre amplitude atual (normalizada) e alvo
    # EN: RMSE between current (normalized) amplitude and target
    amp_current = np.abs(far_shift)
    amp_norm = amp_current / (amp_current.max() if amp_current.max() > 0 else 1.0)
    err = np.sqrt(np.mean((amp_norm - target_amp)**2))
    errors.append(err)

    # ---------------------------------------------------------
    # PT: Impõe restrição de amplitude no far-field (mantém fase)
    # EN: Enforce amplitude constraint in far-field (keep phase)
    far_shift = target_amp * np.exp(1j * np.angle(far_shift))

    # PT: Desfaz o shift e volta ao plano da supercélula
    # EN: Undo shift and go back to the supercell plane
    far = np.fft.ifftshift(far_shift)
    plane_field = np.fft.ifft2(far)

    # ---------------------------------------------------------
    # PT: Impõe amplitude unitária (fase-única) no plano da supercélula
    # EN: Enforce unit amplitude (phase-only) at the supercell plane
    plane_field = np.exp(1j * np.angle(plane_field))


### 7. Fase da Supercélula e "Mosaico" da Metassuperfície | Supercell phase and metasurface tiling

**PT — Ideia geral**  
Após a convergência do GS na **supercélula** (tamanho `N_super × N_super`), extraímos o **mapa de fase**  
$\phi_{\text{super}}(x,y) = \arg\{U_{\text{super}}(x,y)\}$ e montamos a **metassuperfície completa** replicando essa fase em mosaico  
(`n_supercells × n_supercells`). Esse padrão **periódico** produz **ordens discretas de difração** no far-field.

Passos:
1. Extrair $\phi_{\text{super}} = \angle\{U\}$;  
2. Construir o “tile” $ \Phi_{\text{full}} = \mathrm{tile}(\phi_{\text{super}})$;  
3. Formar o campo fase-apenas $U_{\text{full}} = e^{i\,\Phi_{\text{full}}}$.

---

<details>
<summary><b>Show English</b></summary>

**EN — Big picture**  
After GS convergence on the **supercell** (`N_super × N_super`), we extract the **phase map**  
$\phi_{\text{super}}(x,y) = \arg\{U_{\text{super}}(x,y)\}$ and build the **full metasurface** by tiling that phase  
(`n_supercells × n_supercells`). Such a **periodic** pattern yields **discrete diffraction orders** in the far-field.

Steps:
1. Extract $\phi_{\text{super}} = \angle\{U\}$;  
2. Build the **tile** $ \Phi_{\text{full}} = \mathrm{tile}(\phi_{\text{super}})$;  
3. Form the phase-only field $U_{\text{full}} = e^{i\,\Phi_{\text{full}}}$.

</details>


In [10]:
# =============================================================================
# PT: Fase da supercélula e mosaico da metassuperfície
# EN: Supercell phase and metasurface tiling
# =============================================================================

# PT: Extração do mapa de fase da supercélula (rad)
# EN: Extract supercell phase map (rad)
supercell_phase = np.angle(plane_field)

# PT: Replica a fase da supercélula em mosaico (n_supercells × n_supercells)
# EN: Tile the supercell phase to build the full metasurface
full_phase = np.tile(supercell_phase, (n_supercells, n_supercells))

# PT: Campo fase-apenas da metassuperfície
# EN: Phase-only full metasurface field
full_field = np.exp(1j * full_phase)


### 8. Far-Field da Metassuperfície | Metasurface far-field

**PT — Intensidade no plano de Fourier**  
Calculamos a **FFT 2D** da metassuperfície puramente de fase e obtemos a **intensidade**.
$I_{\text{far}} = |\mathcal{F}\{U_{\text{full}}\}|^2$. Para visualização, normalizamos e usamos **$\log_{10}$** (evita saturação e revela lóbulos).

---

<details>
<summary><b>Show English</b></summary>

**EN — Far-field intensity**  
We compute the **2D FFT** of the phase-only metasurface and get the **intensity**  
$I_{\text{far}} = |\mathcal{F}\{U_{\text{full}}\}|^2$. For visualization, we normalize and plot in **$\log_{10}$** to avoid saturation and reveal side lobes.

</details>


In [11]:
# =============================================================================
# PT: Far-field da metassuperfície
# EN: Metasurface far-field
# =============================================================================

# PT: Far-field (FFT) e centralização do DC
# EN: Far-field (FFT) and DC centering
FF_full = np.fft.fft2(full_field)
FF_full_shift = np.fft.fftshift(FF_full)

# PT: Intensidade e versão para plot (log10 da intensidade normalizada)
# EN: Intensity and plot-friendly version (log10 of normalized intensity)
I_far = np.abs(FF_full_shift)**2
I_far_plot = np.log10(I_far / (I_far.max() if I_far.max() > 0 else 1.0) + 1e-12)


### 9. Ordens de Difração Propagantes | Propagating diffraction orders and their intensities

**PT — Seleção de ordens $(p, q)$**  
Para uma rede periódica com período físico da supercélula **d**, as ordens de difração estão em frequências espaciais alvo  
$f_x = p/d$, $f_y = q/d$. Uma ordem $(p, q)$ é **propagante** se satisfaz o **cone** $s_x^2 + s_y^2 \le 1$,  
com $s_x = p\,\lambda/d$ e $s_y = q\,\lambda/d$.

Procedimento:
1. Determinar $p_{\max} \approx \lfloor d/\lambda \rfloor$;  
2. Percorrer $(p,q)$; checar propagação via $s_x^2 + s_y^2 \le 1$;  
3. Para ordens válidas, amostrar $I_{\text{far}}$ no ponto $(f_x, f_y)$.

---

<details>
<summary><b>Show English</b></summary>

**EN — Propagating orders $(p, q)$**  
For a periodic array with supercell period **d**, diffraction orders lie at target spatial frequencies  
$f_x = p/d$, $f_y = q/d$. An order $(p, q)$ is **propagating** if it fits the **cone** $s_x^2 + s_y^2 \le 1$,  
with $s_x = p\,\lambda/d$, $s_y = q\,\lambda/d$.

Procedure:
1. Find $p_{\max} \approx \lfloor d/\lambda \rfloor$;  
2. Loop $(p,q)$; check $s_x^2 + s_y^2 \le 1$;  
3. For valid orders, sample $I_{\text{far}}$ at $(f_x, f_y)$.

</details>


In [12]:
# =============================================================================
# PT: Ordens de difração propagantes e suas intensidades
# EN: Propagating diffraction orders and their intensities
# =============================================================================

# PT: p_max ~ floor(d / λ)
# EN: p_max ~ floor(d / λ)
p_max = int(np.floor(d / wavelength + 1e-12))

orders, intensities = [], []

# PT/EN: Eixos completos de frequência (metassuperfície) e centralização
kx_full = np.fft.fftfreq(N_total, d=dx)
ky_full = np.fft.fftfreq(N_total, d=dx)
kx_full_shift = np.fft.fftshift(kx_full)
ky_full_shift = np.fft.fftshift(ky_full)

# PT: Varre ordens inteiras (p,q) dentro do cone s^2 <= 1
# EN: Sweep integer orders (p,q) within the propagating cone s^2 <= 1
for p in range(-p_max, p_max + 1):
    for q in range(-p_max, p_max + 1):
        # PT: Fatores direção (s) para o cone de propagação
        # EN: Direction cosines (s) for the propagation cone
        sx, sy = p * wavelength / d, q * wavelength / d
        if (sx**2 + sy**2) <= 1.0 + 1e-12:
            # PT: Frequências espaciais alvo (fx, fy) para a ordem (p,q)
            # EN: Target spatial frequencies (fx, fy) for order (p,q)
            fx_target, fy_target = p / d, q / d

            # PT: Índices mais próximos no grid FFT centralizado
            # EN: Nearest indices on the centered FFT grid
            ix = np.argmin(np.abs(kx_full_shift - fx_target))
            iy = np.argmin(np.abs(ky_full_shift - fy_target))

            # PT: Amostra intensidade no far-field
            # EN: Sample far-field intensity at the order location
            val = I_far[iy, ix]
            orders.append((p, q))
            intensities.append(val)


### 10. Eficiência de Difração e Uniformidade

**PT — Métricas globais**  
Com as intensidades das ordens **propagantes**, avaliamos:

- **Eficiência de difração** $\eta = \dfrac{\sum I_{p,q}}{\sum I_{\text{far}}}$ — fração de energia que sai em ordens propagantes;  
- **Uniformidade (RMSE normalizado)** w.r.t. a média das intensidades:  
  $$ \mathrm{RMSE} = \sqrt{\frac{1}{M}\sum_i \left(\frac{I_i}{\bar I} - 1\right)^2} $$  
  onde $M$ é o nº de ordens propagantes.

Também organizamos os dados em um **DataFrame** para análises e gráficos.

---

<details>
<summary><b>Show English</b></summary>

**EN — Global metrics**  
From the **propagating** orders intensities we compute:

- **Diffraction efficiency** $\eta = \dfrac{\sum I_{p,q}}{\sum I_{\text{far}}}$;  
- **Uniformity (normalized RMSE)** w.r.t. the mean intensity:  
  $$ \mathrm{RMSE} = \sqrt{\frac{1}{M}\sum_i \left(\frac{I_i}{\bar I} - 1\right)^2} $$  
  where $M$ is the number of propagating orders.

We also build a **DataFrame** for plotting and analysis.

</details>


In [13]:
# =============================================================================
# PT: Eficiência, uniformidade e DataFrame das ordens
# EN: Efficiency, uniformity and orders DataFrame
# =============================================================================

intensities = np.array(intensities, dtype=float)
M = len(intensities)

# PT: Energia total no far-field e eficiência (fração nas ordens propagantes)
# EN: Total far-field energy and efficiency (fraction in propagating orders)
total_energy = I_far.sum()
de_sum = (intensities.sum() / total_energy) if total_energy > 0 else 0.0

# PT: Uniformidade via RMSE normalizado pela média
# EN: Uniformity via normalized RMSE w.r.t. the mean
mean_I = intensities.mean() if M > 0 else 1.0
rmse = float(np.sqrt(np.mean((intensities / (mean_I if mean_I > 0 else 1.0) - 1.0)**2))) if M > 0 else float("nan")

# PT: Tabela de ordens e intensidades
# EN: Orders × intensities table
df = pd.DataFrame(orders, columns=["p", "q"])
df["intensity"] = intensities


### 11. Visualizações Finais (fase, tile, far-field, erro, ordens)

**PT — O que será exibido/salvo**  
1. **Fase da supercélula** (rad);  
2. **Fase “tile” da metassuperfície** (rad);  
3. **Intensidade de far-field** (log$_{10}$);  
4. **Evolução do erro GS** (RMSE por iteração);  
5. **Dispersão das ordens propagantes** $(p, q)$ coloridas por intensidade.

Todos os **títulos/labels** ficam em **português**. As figuras são salvas em `RUN_DIR` com nomes **ricos** contendo os parâmetros do estudo.

---

<details>
<summary><b>Show English</b></summary>

**EN — What we plot/save**  
1. **Supercell phase** (rad);  
2. **Metasurface phase tile** (rad);  
3. **Far-field intensity** (log$_{10}$);  
4. **GS error evolution** (RMSE per iteration);  
5. **Propagating orders scatter** $(p, q)$ colored by intensity.

All titles/labels are in **Portuguese**; figures are saved to `RUN_DIR` with **rich** filenames including study parameters.

</details>


In [17]:
# =============================================================================
# PT: Helpers de salvamento (usa rich_name e RUN_DIR definidos antes)
# EN: Save helpers (use rich_name and RUN_DIR defined earlier)
# =============================================================================
def _save_fig(fig, path: Path, dpi: int = 300) -> None:
    """
    EN:
        Save a Matplotlib figure with tight bounding box and close it to free memory.
    PT:
        Salva uma figura do Matplotlib com bbox compacto e fecha para liberar memória.
    """
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)


# -----------------------------------------------------------------------------
# PT: 1) Mapas de fase e far-field (3 painéis)
# EN: 1) Phase maps and far-field (3 panels)
# -----------------------------------------------------------------------------
fig1 = plt.figure(figsize=(15, 5))

ax = fig1.add_subplot(1, 3, 1)
ax.set_title("Fase da supercélula (rad)")
im = ax.imshow(supercell_phase, cmap="twilight", origin="lower")
fig1.colorbar(im, ax=ax, label="fase (rad)")

ax = fig1.add_subplot(1, 3, 2)
ax.set_title("Fase da metassuperfície (tile)")
im = ax.imshow(full_phase, cmap="twilight", origin="lower")
fig1.colorbar(im, ax=ax, label="fase (rad)")

ax = fig1.add_subplot(1, 3, 3)
ax.set_title("Intensidade do far-field (log10)")
im = ax.imshow(I_far_plot, origin="lower", extent=[-0.5, 0.5, -0.5, 0.5])
fig1.colorbar(im, ax=ax, label="log10 I")

fig1.tight_layout()
_save_fig(fig1, RUN_DIR / f"{rich_name('mapas_fase_difracao')}.png")


# -----------------------------------------------------------------------------
# PT: 2) Evolução do erro (RMSE por iteração)
# EN: 2) Error evolution (RMSE per iteration)
# -----------------------------------------------------------------------------
fig2, ax2 = plt.subplots(figsize=(7, 5))
ax2.plot(errors)
ax2.set_xlabel("Iteração GS")
ax2.set_ylabel("Erro (RMSE)")
ax2.set_title("Evolução do erro (GS)")
ax2.grid(True)
fig2.tight_layout()
_save_fig(fig2, RUN_DIR / f"{rich_name('evolucao_erro')}.png")


# -----------------------------------------------------------------------------
# PT: 3) Mapa de ordens propagantes (p,q) colorido por intensidade
# EN: 3) Propagating orders (p,q) map colored by intensity
# -----------------------------------------------------------------------------
fig3, ax3 = plt.subplots(figsize=(7, 6))
sc = ax3.scatter(df["p"], df["q"], c=df["intensity"], cmap="viridis", s=50, edgecolor="k")
cb = fig3.colorbar(sc, ax=ax3, label="Intensidade (não normalizada)")
ax3.set_xlabel("Ordem p")
ax3.set_ylabel("Ordem q")
ax3.set_title("Ordens de difração propagantes")
ax3.grid(True)
fig3.tight_layout()
_save_fig(fig3, RUN_DIR / f"{rich_name('ordens_propagantes')}.png")


# -----------------------------------------------------------------------------
# PT: (Opcional) Salvar dados em disco para análise posterior
# EN: (Optional) Persist data for later analysis
# -----------------------------------------------------------------------------
np.savetxt(RUN_DIR / f"{rich_name('supercell_phase')}.txt", supercell_phase)
np.savetxt(RUN_DIR / f"{rich_name('full_phase_tile')}.txt", full_phase)
np.savetxt(RUN_DIR / f"{rich_name('I_far')}.txt", I_far)
np.savetxt(RUN_DIR / f"{rich_name('errors_rmse')}.txt", np.array(errors))

df_out = RUN_DIR / f"{rich_name('orders_table')}.csv"
df.to_csv(df_out, index=False)


# -----------------------------------------------------------------------------
# PT: Resumo final no terminal
# EN: Final console summary
# -----------------------------------------------------------------------------
mapas_path = RUN_DIR / f"{rich_name('mapas_fase_difracao')}.png"
erro_path  = RUN_DIR / f"{rich_name('evolucao_erro')}.png"
ordens_path = RUN_DIR / f"{rich_name('ordens_propagantes')}.png"

print("\nFiguras e dados salvos em:", RUN_DIR)
print(f"- mapas_fase_difracao: {mapas_path}")
print(f"- evolucao_erro:       {erro_path}")
print(f"- ordens_propagantes:  {ordens_path}")
print(f"- tabela de ordens:    {df_out}")



Figuras e dados salvos em: c:\Users\vinicius23011\MATLAB\Projects\TCC\results\holography\gs_y\demo_dammannY\2025-11-04T12-15-29Z
- mapas_fase_difracao: c:\Users\vinicius23011\MATLAB\Projects\TCC\results\holography\gs_y\demo_dammannY\2025-11-04T12-15-29Z\mapas_fase_difracao__Y__λ_1064nm__P_520nm__scpix_45px__nsc_10__iter_400__seed_0.png
- evolucao_erro:       c:\Users\vinicius23011\MATLAB\Projects\TCC\results\holography\gs_y\demo_dammannY\2025-11-04T12-15-29Z\evolucao_erro__Y__λ_1064nm__P_520nm__scpix_45px__nsc_10__iter_400__seed_0.png
- ordens_propagantes:  c:\Users\vinicius23011\MATLAB\Projects\TCC\results\holography\gs_y\demo_dammannY\2025-11-04T12-15-29Z\ordens_propagantes__Y__λ_1064nm__P_520nm__scpix_45px__nsc_10__iter_400__seed_0.png
- tabela de ordens:    c:\Users\vinicius23011\MATLAB\Projects\TCC\results\holography\gs_y\demo_dammannY\2025-11-04T12-15-29Z\orders_table__Y__λ_1064nm__P_520nm__scpix_45px__nsc_10__iter_400__seed_0.csv
